In [ ]:
# pip install osmnx networkx folium

In [ ]:
import folium
import osmnx as ox
import networkx as nx
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, concat_ws
import numpy as np

In [ ]:
# Agrupar los datos por identificador de tienda para calcular el total de ventas acumuladas
df_store_aggregated = df.groupBy("customer_id", "neighborhood", "commune", "latitude", "longitude") \
    .agg(_sum("quantity_products").alias("total_store")) \
    .orderBy(col("total_store").desc())

# Mostrar los resultados
df_store_aggregated.show()
df_store_aggregated.count()

In [ ]:
# Convertir el DataFrame de Spark a Pandas para trabajar con OR-Tools y Folium
store_df = df_store_aggregated.toPandas()

# Obtener las coordenadas de las tiendas
locations = store_df[['latitude', 'longitude']].to_numpy()

# Definir el número de vehículos y la ubicación del depósito
num_vehicles = 6
depot = 0

# Calcular la distancia euclidiana entre los puntos
def compute_euclidean_distance_matrix(locations):
    distances = np.linalg.norm(locations[:, np.newaxis] - locations, axis=2)
    return distances

distance_matrix = compute_euclidean_distance_matrix(locations)

# Crear el modelo de datos
def create_data_model():
    data = {}
    data['distance_matrix'] = distance_matrix
    data['num_vehicles'] = num_vehicles
    data['depot'] = depot
    return data

data = create_data_model()

# Crear el gestor de rutas
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])

# Crear el modelo de ruta
routing = pywrapcp.RoutingModel(manager)

# Crear la función de distancia
def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return int(data['distance_matrix'][from_node][to_node])

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Definir el costo de la arcada
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Definir la búsqueda de parámetros
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Resolver el problema
solution = routing.SolveWithParameters(search_parameters)

if solution:
    print("Solution found!")
else:
    print("No solution found!")

# Descargar el grafo de la red de calles de Medellín
G = ox.graph_from_place('Medellin, Colombia', network_type='drive')

# Crear un mapa de Folium centrado en Medellín
map_medellin = folium.Map(location=[6.2442, -75.5812], zoom_start=12)

# Función para encontrar la ruta realista en la red de calles usando OSMnx
def get_route(G, start, end):
    orig_node = ox.distance.nearest_nodes(G, start[1], start[0])
    dest_node = ox.distance.nearest_nodes(G, end[1], end[0])
    route = nx.shortest_path(G, orig_node, dest_node, weight='length')
    return route

# Función para trazar la ruta en el mapa
def plot_solution(manager, routing, solution):
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(locations[node_index])
            index = solution.Value(routing.NextVar(index))
        route.append(locations[depot])
        
        # Trazar las rutas realistas
        for i in range(len(route) - 1):
            route_nodes = get_route(G, route[i], route[i + 1])
            route_coords = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in route_nodes]
            folium.PolyLine(route_coords, color=np.random.choice(['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']), weight=2.5, opacity=1).add_to(map_medellin)

# Trazar las rutas en el mapa
plot_solution(manager, routing, solution)

# Agregar marcadores para las tiendas
for _, row in store_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Store ID: {row['customer_id']}, Sales: {row['total_store']}",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(map_medellin)

# Guardar el mapa en un archivo HTML
map_medellin.save("delivery_routes_medellin.html")
print("Generated map ")